# IMPORT ED AUTENTICAZIONE

In [5]:
import os
import json
import tweepy
import math
import pprint as pp
import networkx as nx
import pandas as pd
import numpy as np
import glob

In [6]:
#Twitter API credentials
api_key = "M5NYCCuMTxLKm4S6DrsK5NqpR"
api_secret = "y3svwwgghPa9l0ieZReuFzuOPlkX92BrW4qfn4kwXAFoWffHzR"
access_token = "951194016212807680-RX1wBOiNZxau9k7nHyOvJk1mXNtyOp6"
access_secret = "Dt5x1PnX7jhsZvUCQq8gqlN0HDMjqVyxN8yb0To57uQEq"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAAa9VwEAAAAAs09Lb1bMGnB4QMwrjBNd9J7NaXQ%3DOki3q56qHVQ2uESPZGHvRO8tFJg7rOPEAjpDcaKCRxVgfh8ulq"

auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)
if(api.verify_credentials):
    print ('Authentication completed successfully!')

client = tweepy.Client(
    consumer_key=api_key,
    consumer_secret=api_secret,
    access_token=access_token,
    access_token_secret=access_secret,
    bearer_token=bearer_token,
    wait_on_rate_limit=True,
    return_type=dict
)


Authentication completed successfully!


# GESTIONE JSON

In [3]:
data_folder = "data_v2"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}


# FOLLOWERS

In [4]:
def findUsers(userlist,mode): #mode: 0 = followers, 1 = following
    quantity=20
    foundlist={}
    l2list = {}

    for user in userlist:
        print(f"Processing user {user}")

        found = []
        l2list[user]=[]

        if mode==0:
            usergroup=tweepy.Cursor(api.followers,screen_name = user ).items(quantity)
        else:
            usergroup=tweepy.Cursor(api.friends,screen_name = user ).items(quantity)

        for item in usergroup:

            json_data = item._json

            found_user = {}
            found_user["id"] = json_data["id"]
            found_user["name"] = json_data["name"]
            found_user["screen_name"] = json_data["screen_name"]
            found_user["location"] = json_data["location"]
            found_user["followers_count"] = json_data["followers_count"]
            found_user["friends_count"] = json_data["friends_count"]
            found_user["protected"] = json_data["protected"]


            if item.friends_count < 1000 and item.followers_count<1000 and json_data["protected"] == False and (found_user not in found):
                found.append(found_user)
                l2list[user].append(item.screen_name)
                print(item.screen_name)

            if len(found) >= 5:
                break

        foundlist[user] = found

        if mode==0:
            serialize_json(data_folder, user+"_followers.json", foundlist)
        else:
            serialize_json(data_folder, user+"_following.json", foundlist)

    return l2list


def findUsersV2(utenti,mode):
    followers = {}
    for utente in utenti:

        page_counter = 0
        data_for_page=10
        data_total=5
        next_token = None

        response = client.get_users(ids=utente)

        for user in response['data']:

            print(f"Processing user with id {user['id']} and username {user['username']}")
            followers_of_user = []

            while (next_token and page_counter<= math.ceil(data_total/data_for_page) or len(followers_of_user)<=0):

                if next_token:
                    if mode==0:
                        response = client.get_users_followers(
                            id=user['id'], user_fields=['name', 'description', 'location','public_metrics','protected'],
                            max_results=data_for_page, pagination_token=next_token
                        )
                    else:
                        response = client.get_users_following(
                            id=user['id'], user_fields=['name', 'description', 'location','public_metrics','protected'],
                            max_results=data_for_page, pagination_token=next_token
                        )
                else:
                    if mode==0:
                        response = client.get_users_followers(
                            id=user['id'], user_fields=['name', 'description', 'location','public_metrics','protected'],
                            max_results=data_for_page
                        )
                    else:
                        response = client.get_users_following(
                            id=user['id'], user_fields=['name', 'description', 'location','public_metrics','protected'],
                            max_results=data_for_page
                        )
                next_token = response['meta']['next_token'] if 'next_token' in response['meta'] else None

                for data in response['data']:
                    if(len(followers_of_user)<data_total and data['public_metrics']['followers_count']<1000 and data['public_metrics']['following_count']<1000 and data['protected']==False):
                        followers_of_user.append(data['id'])

                page_counter=page_counter+1

            followers[user['name']] = followers_of_user
            serialize_json(data_folder, user['name']+"_followers.json", followers)
            #print(f"Found {len(followers[user])} followers for user {user}")

    return followers

#miccighel_id="15750573"
#mizzaro_id="18932422"

#l1_followers=findUsers([mizzaro_id,miccighel_id],0) #Followers di mizzaro e Miccighel
#l1_following=findUsers([mizzaro_id,miccighel_id],1) #Following di mizzaro e Miccighel

#l2a_followers_mizzaro=findUsers(l1_followers["mizzaro"],0) #Followers dei followers di mizzaro
#l2a_following_mizzaro=findUsers(l1_followers["mizzaro"],1) #Following dei followers di mizzaro

#l2a_followers_miccighel=findUsers(l1_followers["Miccighel_"],0) #Followers dei followers di Miccighel
#l2a_following_miccighel=findUsers(l1_followers["Miccighel_"],1) #Following dei followers di Miccighel

#l2b_followers_mizzaro=findUsers(l1_following["mizzaro"],0) #Followers dei following di mizzaro
#l2b_following_mizzaro=findUsers(l1_following["mizzaro"],1) #Following dei following di mizzaro

#l2b_followers_miccighel=findUsers(l1_following["Miccighel_"],0) #Followers dei following di Miccighel
#l2b_following_miccighel=findUsers(l1_following["Miccighel_"],1) #Following dei following di Miccighel

#l1_followers=findUsers(["mizzaro","Miccighel_"],0)
l1_following=findUsers(["mizzaro","Miccighel_"],1)

#l2a_followers_mizzaro=findUsers(l1_followers["mizzaro"],0)
#l2a_following_mizzaro=findUsers(l1_followers["mizzaro"],1)

#l2a_followers_miccighel=findUsers(l1_followers["Miccighel_"],0)
#l2a_following_miccighel=findUsers(l1_followers["Miccighel_"],1)

l2b_followers_mizzaro=findUsers(l1_following["mizzaro"],0)
l2b_following_mizzaro=findUsers(l1_following["mizzaro"],1)

l2b_followers_miccighel=findUsers(l1_following["Miccighel_"],0)
l2b_following_miccighel=findUsers(l1_following["Miccighel_"],1)

fileList=glob.glob(data_folder+"/*.json")

totalData=[]

for fileName in fileList:
    with open(fileName,'r',encoding="UTF-8") as file:
        data=json.load(file)
        totalData.append(data)

serialize_json(data_folder, "total_users.json", totalData)

#with open(data_folder+"/total_users.json","w") as outfile:
#    json.dump(totalData,outfile)

Processing user mizzaro


AttributeError: 'API' object has no attribute 'followers'

# 1 - Scaricate i dettagli del profilo dei followers di primo e secondo livello

In [ ]:
def getUserDetails(users):
    for user in users:
       profile_data = client.get_users(
            usernames=user,
            user_fields=['name', 'description', 'location', 'public_metrics']
            )
       serialize_json(data_folder, user["username"]+"_profile_data.json", profile_data)
       pp.pprint(profile_data)


#CHIAMARE FUNZIONE

# 2 - Scaricate gli ultimi 5 tweet di followers di primo e secondo livello

In [ ]:
def getUserTweets(users):

    page_counter = 0
    data_for_page=10
    data_total=5
    for user in users:
        next_token = None
        tweets = []
        userid=user['id']

        while (next_token and page_counter<= math.ceil(data_total/data_for_page) or len(tweets)<=0):
            if next_token:
                response = client.get_users_tweets(
                    id=userid, tweet_fields=['created_at', 'lang', 'public_metrics'],
                    max_results=data_for_page, pagination_token=next_token
                )
            else:
                response = client.get_users_tweets(
                    id=userid, tweet_fields=['created_at', 'lang', 'public_metrics'],
                    max_results=data_for_page
                )
            next_token = response['meta']['next_token'] if 'next_token' in response['meta'] else None
            for data in response['data']:
                if(len(tweets)<data_total):
                    tweets.append(data)
            page_counter=page_counter+1
        pp.pprint(tweets)
        serialize_json(data_folder, user["username"]+"_recent_tweets.json", tweets)

